In [7]:
import re
import os
import numpy as np
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from numpy import genfromtxt

#%% Import libraries
import torch
import torchmetrics
from torch.utils.data import DataLoader, TensorDataset
import os
from glob import glob
from sklearn.model_selection import train_test_split
import matplotlib.pylab as plt
from torch.optim.lr_scheduler import CosineAnnealingLR

In [8]:
SVD_matricer = ['V300', 'V500', 'A_k200', 'A_k300', 'A_k500']
dimensioner = [300, 500, 200, 300, 500]
num_epochs = 100
learning_rate = 0.001 
weight_decay = 1e-4
batch_size = 128

# Load sentences and labels
train = pd.read_csv(r'C:\Users\chris\Documents\GitHub\Nyt-rstaler\Test_og_Train_df\train_dataframe.csv')
test = pd.read_csv(r'C:\Users\chris\Documents\GitHub\Nyt-rstaler\Test_og_Train_df\test_dataframe.csv')

train_labels = train["Labels"].tolist()
test_labels = test["Labels"].tolist()

train_tf_idf = pd.read_csv(r'C:\Users\chris\Documents\GitHub\Nyt-rstaler\DTTFIDFM_data\DTTFIDFM_train.csv')
test_tf_idf = pd.read_csv(r'C:\Users\chris\Documents\GitHub\Nyt-rstaler\DTTFIDFM_data\DTTFIDFM_test.csv')

train_tf_idf = train_tf_idf.drop('Unnamed: 0', axis=1)
test_tf_idf = test_tf_idf.drop('Unnamed: 0', axis=1)

train_tf_idf_numpy = train_tf_idf.to_numpy()
test_tf_idf_numpy = test_tf_idf.to_numpy()

current_matrix = genfromtxt(f'{SVD_matricer[0]}.csv', delimiter=',') # Load SVD Matrix

# Sentence LSA embedding

def sentence_LSA_embedding(TF_IDF_matrix_numpy, V_k):
    liste_LSA_vektorer = []
    for i in range(TF_IDF_matrix_numpy.shape[0]):
        lsa_vector = np.dot(TF_IDF_matrix_numpy[i], V_k)
        liste_LSA_vektorer.append(lsa_vector)
    return np.array(liste_LSA_vektorer)

num_epochs = 100
learning_rate = 0.001 
weight_decay = 1e-4 

device = "cuda" if torch.cuda.is_available() else "cpu"

FileNotFoundError: V300.csv not found.

# Neural netværk

In [10]:
class NetSS:
    def __init__(self, input_dim, con_layers, num_of_lin_lay):
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.input_dim = input_dim   # Initial input dimension
        self.con_layers = con_layers  # Number of convolutional layers
        self.num_of_lin_lay = num_of_lin_lay  # Number of linear layers
    
    def conv_block(self):
        """Creates a convolutional block based on `con_layers`."""
        layers = []
        current_dim = self.input_dim
        
        if self.con_layers >= 1:
            layers.append(torch.nn.Conv1d(1, 16, kernel_size=3))  # Conv 1
            current_dim = (current_dim - 3 + 1) // 2  # After pooling
            layers.append(torch.nn.ReLU())
            layers.append(torch.nn.Dropout(p=0.05))
            layers.append(torch.nn.MaxPool1d(kernel_size=2))
        
        if self.con_layers >= 2:
            layers.append(torch.nn.Conv1d(16, 16, kernel_size=3))  # Conv 2
            current_dim = (current_dim - 3 + 1) // 2  # After pooling
            layers.append(torch.nn.ReLU())
            layers.append(torch.nn.Dropout(p=0.05))
            layers.append(torch.nn.MaxPool1d(kernel_size=2))
        
        layers.append(torch.nn.Flatten())
        self.output_dim = 16 * current_dim  # Update for final linear input dimension
        return torch.nn.Sequential(*layers)

    def linear_layers(self):
        """Creates the linear layers dynamically based on `num_of_lin_lay`."""
        layers = []
        current_dim = self.output_dim

        for i in range(self.num_of_lin_lay - 1):
            next_dim = current_dim // 2
            layers.append(torch.nn.Linear(current_dim, next_dim))
            layers.append(torch.nn.ReLU())
            current_dim = next_dim
        
        # Final output layer
        layers.append(torch.nn.Linear(current_dim, 2))
        return torch.nn.Sequential(*layers)

    def build_model(self):
        """Builds the model based on `con_layers` and `num_of_lin_lay`."""
        model = torch.nn.Sequential(
            self.conv_block(),
            self.linear_layers()
        )
        return model.to(self.device)

model = NetSS(dimensioner[0], 1, 1)
net = model.build_model()

In [11]:
class Load:
    def __init__(self,the_matrix):
        self.the_matrix = the_matrix

        train_lsa = sentence_LSA_embedding(train_tf_idf_numpy, the_matrix)
        test_lsa = sentence_LSA_embedding(test_tf_idf_numpy, the_matrix)

        #Tensors

        # Convert inputs
        train_input_tensor = torch.tensor(train_lsa, dtype=torch.float32)  
        test_input_tensor = torch.tensor(test_lsa, dtype=torch.float32)   

        # Convert labels
        train_labels_tensor = torch.tensor(train_labels, dtype=torch.long)  # Shape: (num_train_sentences,)
        test_labels_tensor = torch.tensor(test_labels, dtype=torch.long)    # Shape: (num_test_sentences,)

        # Reshape the tensors for CNN input (adding the channel dimension)
        train_input_tensor = train_input_tensor.unsqueeze(1)  # Shape: (num_train_sentences, 1, 200)
        test_input_tensor = test_input_tensor.unsqueeze(1)    # Shape: (num_test_sentences, 1, 200)

        # Create datasets
        train_dataset = TensorDataset(train_input_tensor, train_labels_tensor)
        test_dataset = TensorDataset(test_input_tensor, test_labels_tensor)

        # Create dataloaders
        self.train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)  # Shuffle training data
        self.test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)   # No shuffle for testing

        self.loss_function = torch.nn.CrossEntropyLoss()
        self.optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate, weight_decay=weight_decay)

In [18]:
#%% Train
class Train:
    def __init__(self, matrix):
        self.matrix = matrix
        pass

        train_loss = {}
        test_loss = {}
        train_accuracy = {}
        test_accuracy = {}
        step = 0
        Loader = Load(self.matrix)

        #scheduler = CosineAnnealingLR(optimizer, T_max=num_epochs)

        for epoch in range(num_epochs):
            running_loss = 0.0
            correct = 0
            total = 0

            for inputs, labels in Loader.train_loader:
                net.train()

                # Put data on GPU 
                inputs = inputs.to(device)
                labels = labels.to(device)

                Loader.optimizer.zero_grad()

                # Compute loss and take gradient step
                outputs = net(inputs)
                loss = Loader.loss_function(outputs, labels)

                loss.backward()
                Loader.optimizer.step()

                predicted = torch.argmax(outputs, dim=1)

                running_loss += loss.item()
                correct += (predicted == labels).sum().item()
                total += labels.size(0)

                # Print accuracy for epoch            
            epoch_loss = running_loss / len(Loader.train_loader)

            train_loss[epoch] = epoch_loss

            epoch_accuracy = 100 * correct / total

            train_accuracy[epoch] = epoch_accuracy

            print(f"Epoch {epoch+1}/{num_epochs}")
            print(f"Training loss: {epoch_loss:.4f}, Training accuracy: {epoch_accuracy:.2f}%")

            #tre_loss = epoch_loss

            # Evaluate the model
            net.eval()
            correct = 0
            total = 0
            running_loss_test = 0.0

            with torch.no_grad():
                for inputs, labels in Loader.test_loader:
                    # Put data on GPU 
                    inputs = inputs.to(device)
                    labels = labels.to(device)
                    
                    # Forward pass
                    outputs = net(inputs)
                    
                    predicted = torch.argmax(outputs, dim=1)

                    loss = Loader.loss_function(outputs, labels)
                    
                    running_loss_test += loss.item()

                    # Count correct predictions
                    correct += (predicted == labels).sum().item()
                    total += labels.size(0)

                epoch_loss = running_loss_test / len(Loader.test_loader)
                test_loss[epoch] = epoch_loss

            test_accuracy_score = 100 * correct / total
            
            test_accuracy[epoch] = test_accuracy_score

            print(f"Test loss: {epoch_loss:.4f}, Test accuracy: {test_accuracy_score:.2f}%")
            print()

            #scheduler.step()
            
            #val_loss = epoch_loss
            #if (val_loss - tre_loss) > 0.5:
                #break

In [21]:
Train(current_matrix)

KeyboardInterrupt: 